In [2]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import to_monthly

In [4]:
# Parameters
vname = 'sst'
grid = '2x2'
lat_slice = (-10, 10)
lon_slice = (120, 290)
data_dir = '../data/real'
library_dir = '../data/cesm2'

# load target data
f = f'{data_dir}/{vname}_anomaly_{grid}.nc'
da = xr.open_dataarray(f)
da = da.sel(lat=slice(*lat_slice), lon=slice(*lon_slice))
da_month = to_monthly(da)

# Load library
f = f'{library_dir}/{vname}_anomaly_{grid}.nc'
library = xr.open_dataarray(f)
library = library.sel(lat=slice(*lat_slice), lon=slice(*lon_slice))
library_month = to_monthly(library)
library_month = library_month.rename({'year': 'lyear', 'ens': 'lens'})

In [5]:
# Grid weights
wgt = np.cos(np.deg2rad(da.lat))

In [7]:
%%time
msd = ((da_month - library_month) ** 2).weighted(wgt).mean(dim=['lat', 'lon'])

CPU times: user 24.9 s, sys: 5.58 s, total: 30.5 s
Wall time: 30.7 s


In [8]:
# Save
encoding = {vname: {'dtype': 'float32'}}
outf = f'{data_dir}/target_distance.nc'
msd.to_netcdf(outf, encoding=encoding)

In [9]:
msd

<xarray.DataArray 'sst' (year: 37, month: 12, lens: 100, lyear: 136)>
array([[[[ 0.46859594,  0.45919112,  4.46164564, ...,  0.46620564,
           0.2557932 ,  1.75425423],
         [ 0.30316445,  4.6293548 ,  1.12587666, ...,  1.84526197,
           0.59178177,  2.64886752],
         [ 1.64700755,  0.59527658,  0.29329755, ...,  4.95602052,
           1.74152122,  0.29772763],
         ...,
         [ 0.43776631,  1.58482515,  4.5321756 , ...,  0.38749167,
           3.86325791,  1.77990935],
         [ 1.44163831,  1.6571567 ,  0.72055196, ...,  0.50608609,
           2.49585504,  3.87577256],
         [ 0.65119696,  0.22197216,  2.46672368, ...,  2.56181341,
           0.21956898,  2.58752313]],

        [[ 0.39998522,  0.68648862,  4.41348932, ...,  0.43389932,
           0.21908924,  1.31206726],
         [ 0.40603956,  4.11795877,  0.91263851, ...,  1.37617206,
           0.52128253,  2.40523943],
         [ 1.51903193,  0.60016493,  0.29051868, ...,  4.39063009,
           1.53259151,  0.32570809],
...
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * year     (year) int64 1987 1988 1989 1990 1991 ... 2019 2020 2021 2022 2023
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * lyear    (lyear) int64 1865 1866 1867 1868 1869 ... 1996 1997 1998 1999 2000
  * lens     (lens) int64 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100